<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


# Assistant with Multiple Files

Let's pretend you just got hired to the executive team at a new company: ACME Dog Food Corporation. To help answer questions, let's create an assistant with access to a knowledge base of multiple corporate documents.

In [1]:
from openai import OpenAI

In [2]:
client = OpenAI()

## Uploading Files

In [3]:
import os

In [4]:
os.listdir('02-Multiple-Files/')

['ACME Advertising Policy.pdf',
 'ACME_Dog_Food_Ingredients.pdf',
 'Corporate Travel Policy.pdf']

In [5]:
files_to_upload = ['ACME Advertising Policy.pdf',
 'ACME_Dog_Food_Ingredients.pdf',
 'Corporate Travel Policy.pdf']

In [6]:
def upload_assistant_file(filename):
    file = client.files.create(
      file=open('02-Multiple-Files/'+filename, "rb"),
      purpose='assistants'
    )
    print(file.id)

In [7]:
for file in files_to_upload:
    upload_assistant_file(file)

file-AoLOXQ2kfxZjcLM3mXzBc8Z6
file-0rfUyiB2dyurS2qIHVLf72lL
file-HYr6rYHNvdyz7JODyJ0jzrr7


In [8]:
for file in client.files.list():
    print(f"{file.filename} has id of: {file.id}")

Corporate Travel Policy.pdf has id of: file-HYr6rYHNvdyz7JODyJ0jzrr7
ACME_Dog_Food_Ingredients.pdf has id of: file-0rfUyiB2dyurS2qIHVLf72lL
ACME Advertising Policy.pdf has id of: file-AoLOXQ2kfxZjcLM3mXzBc8Z6


## Adding Files for the Assistant 

1. **Assistant-Level Attachment**: When you attach a file at this level, it becomes a part of the Assistant's general knowledge base. This means the Assistant can access this information when responding to any query.

In [58]:
# Add the file to the assistant
assistant = client.beta.assistants.create(
  instructions="You answer information about the company based on the PDF documents available in your knowledge base.",
  model="gpt-3.5-turbo-1106", # CHECK FOR ERROR IF YOU PROVIDE THE WRONG MODEL
  tools=[{"type": "retrieval"}],
  file_ids= ['file-AoLOXQ2kfxZjcLM3mXzBc8Z6',
'file-0rfUyiB2dyurS2qIHVLf72lL',
'file-HYr6rYHNvdyz7JODyJ0jzrr7'])

## Run a Thread

In [59]:
thread = client.beta.threads.create()

In [60]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Does our dog food allow for artificial ingredients?"
)

In [61]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [63]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id,
)
print(run.status)

completed


Once the run is completed, let's get the messages

In [64]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc",
)

In [68]:
for message in messages:
    print(message)
    print('\n')

ThreadMessage(id='msg_Sf2OOYDbI9WxSkQo3rVt5MWb', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Does our dog food allow for artificial ingredients?'), type='text')], created_at=1702319885, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_xnZzIhY5R4AAQOMTjjEM3ZDN')


ThreadMessage(id='msg_h0sTfwjVcoOPc0PCbobA51Qe', assistant_id='asst_lbz8tHzLjrR84E1NvZOWB4Az', content=[MessageContentText(text=Text(annotations=[], value='The dog food produced by ACME Dog Food Corporation exclusively contains naturally sourced ingredients, including meats, vegetables, grains, and fruits. Importantly, they strictly prohibit the use of artificial colors, flavors, or preservatives in any of their products【13†source】.'), type='text')], created_at=1702319895, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_1jkBqbcCff7clqEX6MbYa8Tb', thread_id='thread_xnZzIhY5R4AAQOMTjjEM3ZDN')




In [65]:
def display_thread_messages(messages):
    # EXPECTS MESSAGES IN ASC ORDER!
    for thread_message in messages.data:
        print(thread_message.content[0].text.value)
        print('\n')

In [66]:
display_thread_messages(messages)

Does our dog food allow for artificial ingredients?


The dog food produced by ACME Dog Food Corporation exclusively contains naturally sourced ingredients, including meats, vegetables, grains, and fruits. Importantly, they strictly prohibit the use of artificial colors, flavors, or preservatives in any of their products【13†source】.




### Important Note on Citations: 

As of this filming/creation of the notebook, annotations are still often empty, but you should be able to access them as described in the documentation at some point in the future:

https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages

For example:

    # Retrieve the message object
    message = client.beta.threads.messages.retrieve(
      thread_id="...",
      message_id="..."
    )

    # Extract the message content
    message_content = message.content[0].text
    annotations = message_content.annotations
    citations = []

    # Iterate over the annotations and add footnotes
    for index, annotation in enumerate(annotations):
        # Replace the text with a footnote
        message_content.value = message_content.value.replace(annotation.text, f' [{index}]')

        # Gather citations based on annotation attributes
        if (file_citation := getattr(annotation, 'file_citation', None)):
            cited_file = client.files.retrieve(file_citation.file_id)
            citations.append(f'[{index}] {file_citation.quote} from {cited_file.filename}')
        elif (file_path := getattr(annotation, 'file_path', None)):
            cited_file = client.files.retrieve(file_path.file_id)
            citations.append(f'[{index}] Click <here> to download {cited_file.filename}')
            # Note: File download functionality not implemented above for brevity

    # Add footnotes to the end of the message before displaying to user
    message_content.value += '\n' + '\n'.join(citations)

# Ask another message

In [86]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Are employees allowed to take Uber and Lyft for corporate travel?"
)

In [87]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [89]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id,
)
print(run.status)

completed


In [90]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc",
)

display_thread_messages(messages)

Does our dog food allow for artificial ingredients?


The dog food produced by ACME Dog Food Corporation exclusively contains naturally sourced ingredients, including meats, vegetables, grains, and fruits. Importantly, they strictly prohibit the use of artificial colors, flavors, or preservatives in any of their products【13†source】.


Are employees allowed to take Uber and Lyft for corporate travel?


Yes, employees are allowed to use Uber Pool or Lyft Line for corporate travel, and the cost is reimbursable up to $50 per day. Uber Pool and Lyft Line are actually preferred options for ridesharing according to the corporate travel policy【17†source】.




----

# Optional: Delete Assistant and All Files

In [92]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
response = client.beta.assistants.delete(my_assistants.data[0].id)
print(response)

AssistantDeleted(id='asst_lbz8tHzLjrR84E1NvZOWB4Az', deleted=True, object='assistant.deleted')


In [93]:
for file in client.files.list():
    client.files.delete(file.id)

In [94]:
client.files.list()

SyncPage[FileObject](data=[], object='list', has_more=False)